### In this project, we would try to explore regression analysis and make predictions on used car prices using ensemble of regressors. I also hope that I can upload the model as a web app for other people to play around with.

**Citations:**

URL: https://www.kaggle.com/austinreese/craigslist-carstrucks-data

---

The following description is from Kaggle:

**Context**

Craigslist is the world's largest collection of used vehicles for sale, yet it's very difficult to collect all of them in the same place. I built a scraper for a school project and expanded upon it later to create this dataset which includes every used vehicle entry within the United States on Craigslist.

**Content**

This data is scraped every few months, it contains most all relevant information that Craigslist provides on car sales including columns like price, condition, manufacturer, latitude/longitude, and 18 other categories. For ML projects, consider feature engineering on location columns such as long/lat. For previous listings, check older versions of the dataset.

See https://github.com/AustinReese1998/craigslistFilter

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
#data import
df = pd.read_csv('Data/vehicles.csv')
df.head()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,vin,drive,size,type,paint_color,image_url,description,county,state,lat,long
0,7088746062,https://greensboro.craigslist.org/ctd/d/cary-2...,greensboro,https://greensboro.craigslist.org,10299,2012.0,acura,tl,NaN,NaN,gas,90186.0,clean,automatic,19UUA8F22CA003926,NaN,NaN,other,blue,https://images.craigslist.org/01414_3LIXs9EO33...,2012 Acura TL Base 4dr Sedan Offered by: B...,NaN,nc,35.7636,-78.7443
1,7088745301,https://greensboro.craigslist.org/ctd/d/bmw-3-...,greensboro,https://greensboro.craigslist.org,0,2011.0,bmw,335,NaN,6 cylinders,gas,115120.0,clean,automatic,NaN,rwd,NaN,convertible,blue,https://images.craigslist.org/00S0S_1kTatLGLxB...,BMW 3 Series 335i Convertible Navigation Dakot...,NaN,nc,NaN,NaN
2,7088744126,https://greensboro.craigslist.org/cto/d/greens...,greensboro,https://greensboro.craigslist.org,9500,2011.0,jaguar,xf,excellent,NaN,gas,85000.0,clean,automatic,NaN,NaN,NaN,NaN,blue,https://images.craigslist.org/00505_f22HGItCRp...,2011 jaguar XF premium - estate sale. Retired ...,NaN,nc,36.1032,-79.8794
3,7088743681,https://greensboro.craigslist.org/ctd/d/cary-2...,greensboro,https://greensboro.craigslist.org,3995,2004.0,honda,element,NaN,NaN,gas,212526.0,clean,automatic,5J6YH18314L006498,fwd,NaN,SUV,orange,https://images.craigslist.org/00E0E_eAUnhFF86M...,2004 Honda Element LX 4dr SUV Offered by: ...,NaN,nc,35.7636,-78.7443
4,7074612539,https://lincoln.craigslist.org/ctd/d/gretna-20...,lincoln,https://lincoln.craigslist.org,41988,2016.0,chevrolet,silverado k2500hd,NaN,NaN,gas,NaN,clean,automatic,1GC1KWE85GF266427,NaN,NaN,NaN,NaN,https://images.craigslist.org/00S0S_8msT7RQquO...,"Shop Indoors, Heated Showroom!!!www.gretnaauto...",NaN,ne,41.1345,-96.2458


It seems the data has some NULL values for some columns, let's view the info for df

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539759 entries, 0 to 539758
Data columns (total 25 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            539759 non-null  int64  
 1   url           539759 non-null  object 
 2   region        539759 non-null  object 
 3   region_url    539759 non-null  object 
 4   price         539759 non-null  int64  
 5   year          538772 non-null  float64
 6   manufacturer  516175 non-null  object 
 7   model         531746 non-null  object 
 8   condition     303707 non-null  object 
 9   cylinders     321264 non-null  object 
 10  fuel          536366 non-null  object 
 11  odometer      440783 non-null  float64
 12  title_status  536819 non-null  object 
 13  transmission  535786 non-null  object 
 14  vin           315349 non-null  object 
 15  drive         383987 non-null  object 
 16  size          168550 non-null  object 
 17  type          392290 non-null  object 
 18  pain

#### We would start by dropping some of the columns. For example, columns that contain too many NULL values to be useful or columns that are hard to make use of for this analysis

In [5]:
#columns contain too many NULL values
df.drop(['condition','cylinders', 'vin', 'drive', 'size', 'type', 'paint_color', 'county'], axis = 1, inplace = True)
#columns that are hard to make use of
df.drop(['id', 'url', 'region', 'region_url', 'image_url', 'description', 'lat', 'long'], axis = 1, inplace = True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539759 entries, 0 to 539758
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   price         539759 non-null  int64  
 1   year          538772 non-null  float64
 2   manufacturer  516175 non-null  object 
 3   model         531746 non-null  object 
 4   fuel          536366 non-null  object 
 5   odometer      440783 non-null  float64
 6   title_status  536819 non-null  object 
 7   transmission  535786 non-null  object 
 8   state         539759 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 37.1+ MB


#### We still have some NULL values in some of the columns left - note that column "odometer" still contains many NULL values, because I think it is an important factor in predicting used car prices, and because we have enough number of examples to work with, we would drop the rows where "odometer" is NULL.

In [7]:
df.drop(df.index[df['odometer'].isna()], inplace = True)
df.reset_index(drop = True, inplace = True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440783 entries, 0 to 440782
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   price         440783 non-null  int64  
 1   year          439815 non-null  float64
 2   manufacturer  425660 non-null  object 
 3   model         436331 non-null  object 
 4   fuel          437477 non-null  object 
 5   odometer      440783 non-null  float64
 6   title_status  437924 non-null  object 
 7   transmission  437038 non-null  object 
 8   state         440783 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 30.3+ MB


#### Now we still have some missing values but they are a small percentage of the overall number of records. We can impute the missing values from existing data.

In [9]:
#for categorical variables, we would impute missing values based on existing class distribution by random sampling with replacement
def CategoricalImputer(df, column_names):
    for column in column_names:
        replacements = np.random.choice(a = df[column][df[column].notna()], size = df[column].isna().sum(), replace = True)
        replacements = pd.Series(replacements, index = df.index[df[column].isna()])
        df[column].fillna(replacements, inplace = True)

In [10]:
CategoricalImputer(df, ['year', 'manufacturer', 'model', 'fuel', 'title_status', 'transmission'])

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440783 entries, 0 to 440782
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   price         440783 non-null  int64  
 1   year          440783 non-null  float64
 2   manufacturer  440783 non-null  object 
 3   model         440783 non-null  object 
 4   fuel          440783 non-null  object 
 5   odometer      440783 non-null  float64
 6   title_status  440783 non-null  object 
 7   transmission  440783 non-null  object 
 8   state         440783 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 30.3+ MB


#### Great, we do not have NULL values anymore, now we would like to perform some data validity checking

In [12]:
print(f"The max price of a used car is {df['price'].max()}, the min price of a used car is {df['price'].min()}.")

The max price of a used car is 4198286601, the min price of a used car is 0.


In [13]:
#The above does not make lots of sense, so we would like to only keep the used cars with prices between 1000 and 200000
drop_idx = df.index[(df['price'] < 1000) | (df['price'] > 200000)]
df.drop(drop_idx, inplace = True)
df.reset_index(drop = True, inplace = True)

In [14]:
print(f"The max price of a used car is {df['price'].max()}, the min price of a used car is {df['price'].min()}.")

The max price of a used car is 200000, the min price of a used car is 1000.


In [15]:
df['year'].value_counts()

2017.0    32514
2016.0    30970
2015.0    30277
2013.0    29290
2014.0    28776
          ...  
1926.0        2
1925.0        2
1922.0        1
1914.0        1
0.0           1
Name: year, Length: 101, dtype: int64

In [16]:
#We would only be interested in used cars with year between 2000 and 2019. 
#If a used car is too old, we are not intersted and also they have fewer instances to be trained thus prediction accuracy is questionable.
#The dataset is from Jan 2020 so we would only include models up until year 2019 
#(year 2020 model should be out already but again, there are few instances or even bad data as there are some used cars have model year 2021)
drop_idx = df.index[(df['year'] < 2000) | (df['year'] > 2019)]
df.drop(drop_idx, inplace = True)
df.reset_index(drop = True, inplace = True)

In [17]:
#top 5 rows
df['year'].value_counts().head()

2017.0    32514
2016.0    30970
2015.0    30277
2013.0    29290
2014.0    28776
Name: year, dtype: int64

In [18]:
df['manufacturer'].value_counts()

ford               70466
chevrolet          55187
toyota             30346
nissan             21692
ram                19682
honda              19598
jeep               18601
gmc                17468
dodge              12948
bmw                11464
hyundai             9779
subaru              9550
mercedes-benz       8810
volkswagen          8439
kia                 7476
chrysler            6293
cadillac            5846
buick               5462
lexus               5050
audi                4913
mazda               4793
infiniti            3306
acura               3112
lincoln             2490
volvo               2474
pontiac             2227
mini                2049
mitsubishi          1933
rover               1586
saturn              1184
mercury             1183
jaguar               800
fiat                 788
tesla                203
harley-davidson      157
alfa-romeo            41
ferrari               37
aston-martin          28
land rover            15
porche                 7


In [19]:
#Manufacturer data looks good but for the same reasons as above, we would like to drop instances where count < 1000
counts = df['manufacturer'].value_counts()
values_to_drop = counts.index[counts < 1000]
drop_idx = df.index[df['manufacturer'].isin(values_to_drop)]
df.drop(drop_idx, inplace = True)
df.reset_index(drop = True, inplace = True)

#### The next column: "model" is a hard one to deal with. If we look at its value_counts...

In [20]:
df['model'].value_counts()

f-150                             8155
silverado 1500                    5242
1500                              4690
silverado                         3545
wrangler                          2781
                                  ... 
fordf350 4x4 larait diesel           1
rlawd                                1
silverado 3500 duramax diesel        1
travers lt                           1
silverado 2500hd lt1 automatic       1
Name: model, Length: 25488, dtype: int64

#### Noticed the issue here? Because this "model" field might be a free text field, seller can put different variations of the same car model. For example, "silverado 1500", "1500" and "silverado" most likely represent the same car model. If we put the values directly into training algorithms, they would be treated as different car models, which could impact our model performance. To mitigate this issue, we would take the following steps:
1. Strip out the words of individual car model for each record
2. Find out the top 150 words based on frequency with weights assigned based on word sequence in a record
    (ex. "silverado 1500" - "silverado" would have more weights than "1500" because the further the word is to the front, the more likely the word represents the actual car model)
3. Go through the 150 words, clean them up and end up with the most common derived "car models"
4. Map records to the most common derived "car models" and mark "N/A" if cannot be mapped

In [21]:
#First we would like to turn all models to lower case for matching later
df['model'] = df['model'].str.lower()

In [22]:
weighted_words = []
for item in df['model']:
    #split into words
    words = item.split()
    #find number of words in list and assign weights
    for word, weight in zip(words, (np.arange(len(words)) + 1)[::-1]):
        weighted_words += [word] * weight

In [23]:
#Find "top models"
top_models = pd.Series(weighted_words).value_counts().head(150)

In [24]:
#Now we would go through all the "top models" and clean the Series up
pd.set_option('display.max_rows', None)
top_models

1500           45134
silverado      43559
sport          28931
cab            27756
sierra         23734
super          22782
f150           22285
crew           21898
grand          21079
4x4            19370
2500           16711
f-150          15449
se             14451
lt             14377
duty           13845
sedan          13622
wrangler       13175
4d             10928
tacoma         10756
xlt            10326
f-250          10251
escape         10155
cherokee        9846
awd             9725
coupe           9650
s               9512
civic           9430
f250            9301
tundra          9166
limited         9118
f-350           9084
utility         8917
accord          8806
mustang         8756
3500            8376
supercrew       7980
equinox         7861
slt             7850
rogue           7462
altima          7455
camry           7445
lx              7096
focus           7053
4dr             6853
explorer        6534
fusion          6440
2500hd          6242
camaro       

In [25]:
top_models.drop(['sport', 'cab', 'super', '4x4', 'se', 'lt', 'sedan', 'duty', '4d', 'xlt', 
                'coupe', 'awd', 's', 'limited', 'utility', 'slt', 'lx', '4dr', '4wd', 
                'premium', 'sv', 'series', 'ls', 'double', 'gas', 'xl', 'unlimited', 'ex',
                '2d', 'pickup', '3', 'le', 'sel', 'gt', 'hd', 'ltz', 'fwd', 'ex-l', 'sle',
                 'lifted','sxt', '2.5', 'fe', '300', 'quad', 'diesel', 'sl', 'van', 'rx', '4', '2.0t', '2.5i'], inplace = True)

In [26]:
pd.reset_option('display.max_rows')

In [27]:
top_models = pd.Series(top_models.index)

In [28]:
top_models

0          1500
1     silverado
2        sierra
3          f150
4          crew
        ...    
93      liberty
94       denali
95       murano
96      patriot
97       legacy
Length: 98, dtype: object

In [29]:
#The list might not be perfect but it is as good as we can get, now finally we would like to map every record to one of the top_models
#First, we would like to assign "N/A" to records that do not have words in top_models
df.loc[~df['model'].str.contains('|'.join(top_models)), ['model']] = 'N/A'

In [30]:
#map to top_models
from tqdm import tqdm
for item in tqdm(top_models):
    df['model'] = df['model'].apply(lambda x: item if item in x else x)

100%|██████████| 98/98 [00:04<00:00, 20.88it/s]


In [44]:
print(f"We successfuly mapped {(df['model'] != 'N/A').sum()} car models out of total of {len(df)} records, which is pretty good.")

We successfuly mapped 269672 car models out of total of 375407 records, which is pretty good.
